In [1]:
import pandas as pd

In [8]:
#read in fluoride df
fluoride_df=pd.read_csv("../Clean_Data/Florida_Fluoride_2023.csv",encoding="ISO-8859-1")

In [9]:
fluoride_df

,PWS ID,PWS Name,Status,County,Population Served,Fluoride Product,Fluoridated,Water Source
0,FL-1030050,BAY COUNTY WATER SYSTEM,Adjusted,Bay (Primary),27398,H2SiF6,Yes,Unknown
1,FL-1030141,"CALLAWAY, CITY OF WATER SYSTEM",Consecutive,Bay (Primary),13652,H2SiF6,Yes,Surface
2,FL-1030148,"SPRINGFIELD, CITY OF WTR. SYS.",Consecutive,Bay (Primary),7259,H2SiF6,Yes,Surface
3,FL-1030435,"LYNN HAVEN, CITY OF WATER SYS.",Consecutive,Bay (Primary),27361,H2SiF6,Yes,Ground
4,FL-1030467,"MEXICO BEACH, CITY OF",Non-Adjusted,Bay (Primary),4754,Natural,No,Ground
...,...,...,...,...,...,...,...,...
1600,FL-6604862,LITTLE SUMTER UTILITIES #1,Non-Adjusted,Sumter (Primary),22695,Natural,No,Ground
1601,FL-6605002,JUMPER CREEK MANOR,Non-Adjusted,Sumter (Primary),45,Natural,No,Ground
1602,FL-6605012,NORTH SUMTER UTILITY,Non-Adjusted,Sumter (Primary),39804,Natural,No,Ground
1603,FL-6605023,ORANGE BLOSSOM UTILITIES INC,Non-Adjusted,Sumter (Primary),300,Natural,No,Ground


In [12]:
#columns to keep
fluoride_subset= fluoride_df[["PWS ID", "Population Served", "Fluoride Product", "Fluoridated", "Water Source"]]

In [217]:
#keep only the id and drop the Florida associated tag
fluoride_subset["PWS ID"]=fluoride_subset["PWS ID"].str.replace("FL-","")
fluoride_subset.rename(columns={"PWS ID": "PWSID"},inplace=True)

C:\Users\shupe\AppData\Local\Temp\ipykernel_23580\4017172308.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fluoride_subset["PWS ID"]=fluoride_subset["PWS ID"].str.replace("FL-","")
C:\Users\shupe\AppData\Local\Temp\ipykernel_23580\4017172308.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fluoride_subset.rename(columns={"PWS ID": "PWSID"},inplace=True)


In [218]:
fluoride_subset

,PWSID,Population Served,Fluoride Product,Fluoridated,Water Source
0,1030050,27398,H2SiF6,Yes,Unknown
1,1030141,13652,H2SiF6,Yes,Surface
2,1030148,7259,H2SiF6,Yes,Surface
3,1030435,27361,H2SiF6,Yes,Ground
4,1030467,4754,Natural,No,Ground
...,...,...,...,...,...
1600,6604862,22695,Natural,No,Ground
1601,6605002,45,Natural,No,Ground
1602,6605012,39804,Natural,No,Ground
1603,6605023,300,Natural,No,Ground


In [219]:
#read in plant flow report
flow_df=pd.read_csv("../Clean_Data/Flow_Data_Report_2023_V1.csv",index_col=0)

In [220]:
#reformat date section
flow_df["REPORTDATE"] = pd.to_datetime(flow_df["REPORTDATE"].astype(str) + '01', format='%Y%m%d')

In [221]:
#find the most recent report for each pws
recent_flow_rows = flow_df.loc[flow_df.groupby("PWSID")["REPORTDATE"].idxmax()]

In [222]:
#columns to keep to merge
recent_flow_kept= recent_flow_rows[["PWSID","OFFICE","DESIGNCAPCITY","PLANTSTATUS","MAX_TREATED","AVG_TREATED"]]

In [223]:
#convert the pwsid to the same dtype
recent_rows_kept['PWSID'] = recent_rows_kept['PWSID'].astype(str)
fluoride_subset['PWSID'] = fluoride_subset['PWSID'].astype(str)

C:\Users\shupe\AppData\Local\Temp\ipykernel_23580\3298553856.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_rows_kept['PWSID'] = recent_rows_kept['PWSID'].astype(str)
C:\Users\shupe\AppData\Local\Temp\ipykernel_23580\3298553856.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fluoride_subset['PWSID'] = fluoride_subset['PWSID'].astype(str)


In [224]:
#merge on pwsids, but only keep rows with relation to the fluoride df
first_combine=recent_rows_kept.merge(fluoride_subset,left_on="PWSID",right_on="PWSID", how="right")

In [225]:
first_combine

,PWSID,OFFICE,DESIGNCAPCITY,PLANTSTATUS,MAX_TREATED,AVG_TREATED,Population Served,Fluoride Product,Fluoridated,Water Source
0,1030050,NWD,60000000.0,ACTIVE,19850000.0,17300000.0,27398,H2SiF6,Yes,Unknown
1,1030141,NWD,1.0,ACTIVE,0.0,0.0,13652,H2SiF6,Yes,Surface
2,1030148,NWD,1.0,ACTIVE,0.0,0.0,7259,H2SiF6,Yes,Surface
3,1030435,NWD,5832000.0,ACTIVE,2772000.0,1605742.0,27361,H2SiF6,Yes,Ground
4,1030467,NWD,1010000.0,ACTIVE,308000.0,247225.0,4754,Natural,No,Ground
...,...,...,...,...,...,...,...,...,...,...
1600,6604862,CD,5760000.0,ACTIVE,3037000.0,2339032.0,22695,Natural,No,Ground
1601,6605002,CD,110000.0,ACTIVE,21400.0,18284.0,45,Natural,No,Ground
1602,6605012,CD,5764000.0,ACTIVE,1043000.0,629968.0,39804,Natural,No,Ground
1603,6605023,CD,1728000.0,ACTIVE,223800.0,177505.0,300,Natural,No,Ground


In [226]:
#read in microbiology report
microbio_report=pd.read_csv("../Clean_Data/Microbiological_Report_2023_V1.csv",index_col=0)

In [227]:
#convert sampledate to datetime
microbio_report["SAMPLEDATE"]=pd.to_datetime(microbio_report["SAMPLEDATE"], format='%d-%b-%y')

In [228]:
#find the most recent sample date per pws
recent_micro_rows=microbio_report.loc[microbio_report.groupby("PWSID")["SAMPLEDATE"].idxmax()]

In [229]:
#columns to keep
recent_micro_kept=recent_micro_rows[["PWSID","SYSTEMTYPE","SYSTEMSTATUS","SAMPLETYPE","COLIFORMPRESENT","SAMPLESREQUIRED","SAMPLESTAKEN","METHOD","CHLORINERESIDUAL"]]

In [230]:
#convert pwsid to string dtype
recent_micro_kept['PWSID']=recent_micro_kept['PWSID'].astype(str)

C:\Users\shupe\AppData\Local\Temp\ipykernel_23580\2018977937.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_micro_kept['PWSID']=recent_micro_kept['PWSID'].astype(str)


In [231]:
#incorporate micro data into the first iteration
second_combine = first_combine.merge(recent_micro_kept,how="left")

In [232]:
second_combine

,PWSID,OFFICE,DESIGNCAPCITY,PLANTSTATUS,MAX_TREATED,AVG_TREATED,Population Served,Fluoride Product,Fluoridated,Water Source,SYSTEMTYPE,SYSTEMSTATUS,SAMPLETYPE,COLIFORMPRESENT,SAMPLESREQUIRED,SAMPLESTAKEN,METHOD,CHLORINERESIDUAL
0,1030050,NWD,60000000.0,ACTIVE,19850000.0,17300000.0,27398,H2SiF6,Yes,Unknown,COMMUNITY,ACTIVE,D,N,30.0,10.0,CLL,0.79
1,1030141,NWD,1.0,ACTIVE,0.0,0.0,13652,H2SiF6,Yes,Surface,COMMUNITY,ACTIVE,D,N,15.0,5.0,CLL,0.64
2,1030148,NWD,1.0,ACTIVE,0.0,0.0,7259,H2SiF6,Yes,Surface,COMMUNITY,ACTIVE,D,N,9.0,5.0,CLL,0.63
3,1030435,NWD,5832000.0,ACTIVE,2772000.0,1605742.0,27361,H2SiF6,Yes,Ground,COMMUNITY,ACTIVE,D,N,30.0,1.0,CLL,1.52
4,1030467,NWD,1010000.0,ACTIVE,308000.0,247225.0,4754,Natural,No,Ground,COMMUNITY,ACTIVE,D,N,7.0,8.0,CLL,0.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1600,6604862,CD,5760000.0,ACTIVE,3037000.0,2339032.0,22695,Natural,No,Ground,COMMUNITY,ACTIVE,D,N,30.0,2.0,MF,1.57
1601,6605002,CD,110000.0,ACTIVE,21400.0,18284.0,45,Natural,No,Ground,COMMUNITY,ACTIVE,D,N,1.0,1.0,MF,1.00
1602,6605012,CD,5764000.0,ACTIVE,1043000.0,629968.0,39804,Natural,No,Ground,COMMUNITY,ACTIVE,D,N,50.0,6.0,MF,1.40
1603,6605023,CD,1728000.0,ACTIVE,223800.0,177505.0,300,Natural,No,Ground,COMMUNITY,ACTIVE,D,N,1.0,1.0,MF,1.49


In [233]:
#read in chemical report data
chemical_report=pd.read_csv("../Clean_Data/Chemical_Report_2023_V1.csv",index_col=0)

In [234]:
chemical_report

,PWSID,OFFICE,MAILINGNAME,SYSTEMTYPE,POPULATION,SYSTEMSTATUS,SUBPARTH,DESIGNCAP,CONSECIND,INSPINIT,...,LOCATIONCODE,ENTRYPOINT,LAB,METHOD,ANALYSISDATE,MDL,RDL,CHLORINERESIDUAL,REMARKS,CHLORINERESIDUAL_BINARY
0,1030050,NWD,BAY COUNTY WATER SYSTEM,COMMUNITY,30642,ACTIVE,Y,60000000,0,LJ,...,L3,N,E83079,UNK,8-Oct-23,0.65666,0.777566,NaN,"W17 - 17TH @ COVE, NW SIDE OF INTERSECT",0
1,1030050,NWD,BAY COUNTY WATER SYSTEM,COMMUNITY,30642,ACTIVE,Y,60000000,0,LJ,...,L3,N,E83079,UNK,9-Nov-23,0.65666,0.777566,NaN,"W17 - 17TH @ COVE, NW SIDE OF INTERSECT",0
2,1030050,NWD,BAY COUNTY WATER SYSTEM,COMMUNITY,30642,ACTIVE,Y,60000000,0,LJ,...,L3,N,E83079,552.3,9-Mar-23,0.90000,0.777566,NaN,W17,0
3,1030050,NWD,BAY COUNTY WATER SYSTEM,COMMUNITY,30642,ACTIVE,Y,60000000,0,LJ,...,L3,N,E83079,552.3,14-Apr-23,0.65666,0.777566,NaN,"W17 - 17TH @ COVE, NW SIDE OF INTERSECT",0
4,1030050,NWD,BAY COUNTY WATER SYSTEM,COMMUNITY,30642,ACTIVE,Y,60000000,0,LJ,...,L2,N,E83079,UNK,8-Oct-23,0.65666,0.777566,NaN,RW21 - 2521 PALO ALTO AVE,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159836,6605038,CD,CSU WTP NO. 1,COMMUNITY,19699,ACTIVE,N,4320000,0,NaN,...,AAA,Y,E53076,UNK,3-Mar-23,0.65666,0.549900,NaN,POE,0
159837,6605038,CD,CSU WTP NO. 1,COMMUNITY,19699,ACTIVE,N,4320000,0,NaN,...,AAA,Y,E53076,UNK,3-Mar-23,0.65666,0.549900,NaN,POE,0
159838,6605038,CD,CSU WTP NO. 1,COMMUNITY,19699,ACTIVE,N,4320000,0,NaN,...,AAA,Y,E53076,UNK,3-Mar-23,0.65666,0.549900,NaN,POE,0
159839,6605038,CD,CSU WTP NO. 1,COMMUNITY,19699,ACTIVE,N,4320000,0,NaN,...,AAA,Y,E53076,UNK,3-Mar-23,0.65666,0.549900,NaN,POE,0


In [235]:
#convert anlaysis to datetime format
chemical_report["ANALYSISDATE"]=pd.to_datetime(chemical_report["ANALYSISDATE"].astype(str), format='%d-%b-%y')

In [236]:
chemical_report['PWSID']=chemical_report['PWSID'].astype(str)

In [237]:
recent_chemical_rows=chemical_report.loc[chemical_report.groupby("PWSID")["ANALYSISDATE"].idxmax()]

In [238]:
kept_chemicals=recent_chemical_rows[["PWSID","SUBPARTH","POPULATION","CONTAMGROUP","CHLORINERESIDUAL_BINARY"]]

In [239]:
third_combine=second_combine.merge(kept_chemicals,how="left")

In [240]:
third_combine

,PWSID,OFFICE,DESIGNCAPCITY,PLANTSTATUS,MAX_TREATED,AVG_TREATED,Population Served,Fluoride Product,Fluoridated,Water Source,...,SAMPLETYPE,COLIFORMPRESENT,SAMPLESREQUIRED,SAMPLESTAKEN,METHOD,CHLORINERESIDUAL,SUBPARTH,POPULATION,CONTAMGROUP,CHLORINERESIDUAL_BINARY
0,1030050,NWD,60000000.0,ACTIVE,19850000.0,17300000.0,27398,H2SiF6,Yes,Unknown,...,D,N,30.0,10.0,CLL,0.79,Y,30642.0,SOC,0.0
1,1030141,NWD,1.0,ACTIVE,0.0,0.0,13652,H2SiF6,Yes,Surface,...,D,N,15.0,5.0,CLL,0.64,Y,15269.0,DBP,0.0
2,1030148,NWD,1.0,ACTIVE,0.0,0.0,7259,H2SiF6,Yes,Surface,...,D,N,9.0,5.0,CLL,0.63,Y,8118.0,DBP,0.0
3,1030435,NWD,5832000.0,ACTIVE,2772000.0,1605742.0,27361,H2SiF6,Yes,Ground,...,D,N,30.0,1.0,CLL,1.52,Y,30601.0,SOC,0.0
4,1030467,NWD,1010000.0,ACTIVE,308000.0,247225.0,4754,Natural,No,Ground,...,D,N,7.0,8.0,CLL,0.54,Y,6180.0,DBP,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1600,6604862,CD,5760000.0,ACTIVE,3037000.0,2339032.0,22695,Natural,No,Ground,...,D,N,30.0,2.0,MF,1.57,N,25538.0,DBP,1.0
1601,6605002,CD,110000.0,ACTIVE,21400.0,18284.0,45,Natural,No,Ground,...,D,N,1.0,1.0,MF,1.00,N,50.0,INOR,0.0
1602,6605012,CD,5764000.0,ACTIVE,1043000.0,629968.0,39804,Natural,No,Ground,...,D,N,50.0,6.0,MF,1.40,N,45214.0,DBP,1.0
1603,6605023,CD,1728000.0,ACTIVE,223800.0,177505.0,300,Natural,No,Ground,...,D,N,1.0,1.0,MF,1.49,N,335.0,INOR,0.0
